# 用户社交数据（user_friends.csv）处理
（只取训练集和测试集中出现的用户ID）

数据来源于Kaggle竞赛：Event Recommendation Engine Challenge，根据
events they’ve responded to in the past
user demographic information
what events they’ve seen and clicked on in our app
用户对某个活动是否感兴趣

竞赛官网：
https://www.kaggle.com/c/event-recommendation-engine-challenge/data

user_friends.csv文件：共2维特征
user：用户ID
friends：以空格隔开的用户好友ID列表

# 导入工具包

In [1]:
import pandas as pd

import numpy as np
import scipy.sparse as ss
import scipy.io as sio

#保存数据
import pickle

from sklearn.preprocessing import normalize

总的用户数目超过训练集和测试集中的用户，
为节省处理时间和内存，先去处理train和test，得到竞赛需要用到的活动和用户
然后对在训练集和测试集中出现过的事件和用户建立新的ID索引
先运行user_event.ipynb,
得到事件列表文件：PE_userIndex.pkl

# 读取之前算好的测试集和训练集中出现过的用户

In [2]:
# 数据地址
dpath = './data/'

In [3]:
#读取训练集和测试集中出现过的事件列表
userIndex = pickle.load(open(dpath+"PE_userIndex.pkl", 'rb'))
n_users = len(userIndex)

print("number of users in train & test :%d" % n_users)

number of users in train & test :3391


# 读取之前用户-活动分数矩阵，将朋友参加活动的影响扩展到用户

In [4]:
#用户-事件关系矩阵
userEventScores = sio.mmread(dpath+"PE_userEventScores")

#后续用于将用户朋友参加的活动影响到用户
eventsForUser = pickle.load(open(dpath+"PE_eventsForUser.pkl", 'rb'))

# user_friends.csv

In [5]:
#读取数据

"""
  找出某用户的那些朋友
  1)如果你有更多的朋友，可能你性格外向，更容易参加各种活动
  2)如果你朋友会参加某个活动，可能你也会跟随去参加一下
"""
 
#用户有多少个朋友
numFriends = np.zeros((n_users))
userFriends = ss.dok_matrix((n_users, n_users))
    
fin = open(dpath + "user_friends.csv", 'r')
#字段：user，friends
fin.readline()                # skip header

#ln = 0
for line in fin:  #对每个用户        
    cols = line.strip().split(",")
    user = str(cols[0])    #user
    
    if user in userIndex.keys():   #该用户在训练集和测试集的用户列表中
        friends = cols[1].split(" ")  #friends
        i = userIndex[user]       #该用户的索引
        numFriends[i] = len(friends) #朋友的数目
        for friend in friends:  #该用户的每个朋友
            str_friend = str(friend)
            if str_friend in userIndex.keys():  #如果朋友也在训练集或测试集中出现
                j = userIndex[str_friend]   #朋友的索引
            
                # the objective of this score is to infer the degree to
                # and direction in which this friend will influence the
                # user's decision, so we sum the user/event score for
                # this user across all training events.
            
                #userEventScores为用户对活动的打分（interested - not interseted）
                #在Users-Events.ipynb中计算好了
                eventsForUser = userEventScores.getrow(j).todense()
            
                #所有朋友参加活动的数量占总活动数量的比率（平均频率）
                score = eventsForUser.sum() / np.shape(eventsForUser)[1]
                userFriends[i, j] += score
                userFriends[j, i] += score
            
fin.close()


In [6]:
numFriends

array([1024., 4358., 1381., ..., 2196.,  130., 1708.])

In [7]:
#用户的朋友数目
# 归一化数组
sumNumFriends = numFriends.sum(axis=0)
numFriends = numFriends / sumNumFriends
sio.mmwrite(dpath+"UF_numFriends", np.matrix(numFriends))

In [8]:
np.matrix(numFriends)

matrix([[2.74429520e-04, 1.16793345e-03, 3.70104656e-04, ...,
         5.88522682e-04, 3.48396852e-05, 4.57739864e-04]])

In [9]:
# 用l2方法归一化数组
userFriends = normalize(userFriends, norm="l2", axis=0, copy=False)
sio.mmwrite(dpath+"UF_userFriends", userFriends)

In [10]:
print(userFriends.getrow(123))